In [96]:
import pandas as pd
import numpy as np
import pdfplumber as scrapper
import PyPDF2
from io import StringIO
import os
import glob
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
ingre = pd.read_csv("data/preprocessed/Ingredients_List.csv")
ingre

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,I-3643,225.0,g,0.001000,0.1837,P-18907
1,I-6026,1000.0,g,1.000000,0.8163,P-18907
2,I-3642,1.0,Kg,1000.000000,0.0002,P-25993
3,I-6026,5.0,Kg,1000.000000,0.0008,P-25993
4,I-1813,125.0,ml,0.033814,37.8788,P-26044
...,...,...,...,...,...,...
3273,I-1152,1.0,CT,1.000000,1.0000,R-68143
3274,I-2141,15.0,g,0.001000,1.0000,R-68143
3275,P-26054,30.0,ml,0.001000,1.0000,R-68143
3276,P-45255,2.0,piece,1.000000,1.0000,R-68143


In [3]:
conv = pd.read_csv("data/preprocessed/Conversions_List.csv")
conv

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,NaN,1.000000,1.0,XXX,1.00,L
1,NaN,0.877193,1.0,1.14L,1.14,L
2,NaN,0.666667,1.0,1.5L,1.50,L
3,NaN,0.571429,1.0,1.75 L,1.75,L
4,NaN,0.500000,1.0,2L,2.00,L
...,...,...,...,...,...,...
265,I-4345,0.009804,1.0,CT,102.00,g
266,I-4345,0.006897,1.0,cup,145.00,g
267,I-9295,0.066667,1.0,slice,15.00,g
268,I-11656,0.004348,1.0,CT,230.00,g


In [4]:
prod = pd.read_csv("data/preprocessed/Products_List.csv")
prod

,ProdId,Description,SalesGroup
0,R-30406,G&G|Croissant|Multigrain|OK,OK - GRAB & GO
1,R-55155,GRL|Breakfast BLT,OK - GRILL KITCHEN BREAKFAST
2,R-44024,GRL|Breakfast Wrap,OK - GRILL KITCHEN BREAKFAST
3,R-50368,GRL|BreakfastCroissant,OK - GRILL KITCHEN BREAKFAST
4,R-50498,GRL|Crepe|Apple Cinnamon,OK - GRILL KITCHEN BREAKFAST
...,...,...,...
218,R-68143,SANDWICH|Chicken Caesar Wrap,OK - SANDWICH KITCHEN EVENTS
219,R-61321,SANDWICH|Grilled Veg,OK - SANDWICH KITCHEN EVENTS
220,R-68075,SANDWICH|Tofu Bahn Mi,OK - SANDWICH KITCHEN
221,R-37630,SIDE|Soup|Miso,OK - SOUP


In [91]:
os.getcwd()

'/Users/jennylee/CFFS-PyCharm'

In [21]:
item = pd.read_csv("data/preprocessed/Items_List.csv")
item

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-43851,AJWAIN 5LB,5.0,lb,454.00,g,SPICES
1,I-43854,AMCHUR POWDER,5.0,lb,454.00,g,SPICES
2,I-4472,AVOCADO MX,20.0,CT,1.00,CT,PRODUCE
3,I-4973,AVOCADO PULP CHUNKY,12.0,bag,454.00,g,PRODUCE
4,I-3133,BASE CHICK CONC LIQ G/FREE,4.0,pak,946.00,ml,FOOD - GROCERY
...,...,...,...,...,...,...,...
551,I-40935,TOFU SILKEN,12.0,each,400.00,g,FOOD - GROCERY
552,I-3681,TOMATO PUREE HEAVY TFC,6.0,LG CAN,2.84,L,FOOD - GROCERY
553,I-9341,"TORTILLA FLOUR 6""",24.0,pak,12.00,CT,BAKED GOODS
554,I-1152,TORTILLA FLOUR W/ WHEAT 12IN,12.0,pak,10.00,CT,BAKED GOODS


In [30]:
item.loc[item["ItemId"] == "I-61648"]

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
156,I-61648,BEYOND MEAT SAUSAGE VEG ORIG,50.0,each,1.0,ea,MEAT


In [19]:
item.loc[item["ItemId"] == "I-37139"]

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup


---

In [23]:
item_num = 0
products_list = []
item_prod_relations = {}
prod_num_in_items = 0

path = "/Users/jennylee/CFFS-PyCharm"
os.chdir(path)
filepath = glob.glob(os.path.join(os.getcwd(), "notebooks", "data", "AMS", "*.pdf"))

col_list = ["ItemId", "Description", "Qty", "UOM", "PrepId"]
df = pd.DataFrame(columns=col_list)
df

for file in filepath:

    def generate_items(pdf):
        file = open(str(pdf), "rb")
        pdfReader = PyPDF2.PdfFileReader(file)
        pageObj = pdfReader.getPage(0)
        myfile = pageObj.extractText()
        product = myfile.partition('\n')[0]
        products_list.append(product)
        myfile = myfile.split("\n", 4)[4]
        myfile = myfile.splitlines()
        df = pd.read_csv(StringIO("\n". join(myfile)), sep=" ", names=range(20), on_bad_lines="skip", quotechar=None, quoting=3)

        amounts = df.iloc[:, 0:4]
        amounts_col = ["Single", "Double", "Triple", "Half"]
        amounts.columns = amounts_col
        ingredient = df.iloc[:, 5:]
        ingredient["Description"] = ingredient.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
        ingredient["Description"] = ingredient["Description"].str.replace("nan","")
        ingredient['Description'] = ingredient['Description'].str.extract('(\d*\.*\d*\s*[A-Z].*$)')
        df = pd.concat([amounts, ingredient["Description"]], axis=1)

        df["ItemId"] = ""

        df[["Amount", "Unit"]] = df["Single"].str.extract(pat=r"(\d+\.?\d*)(.*)")
        ind_num = df[df["Amount"].isnull()].index.tolist()
        min_nan = min(ind_num)
        df = df.iloc[:min_nan]
        df["InventoryGroup"] = ""

        global prod_num_in_items

        df["PrepId"] = "P-" + str(prod_num_in_items)
        prod_num_in_items += 1
        df = df.rename(columns={"Amount": "Qty", "Unit":"UOM"})

        for ind, row in df.iterrows():

            global item_num
            df.loc[ind, "ItemId"] = "I-" + str(item_num)
            item_num += 1

#         df["PakUOM"] = df["CaseUOM"]
        item_col_list = ["ItemId", "Description", "Qty", "UOM", "PrepId"]
        df = df[item_col_list]

        return df
    
    temp_df = generate_items(file)
    global df
    df = pd.concat([df, temp_df], axis=0)
    
ams_items = df    
ams_items.head()

/var/folders/c5/pphx1c0n3lb12sy85_ybqxqm0000gn/T/ipykernel_28708/2816357255.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ingredient["Description"] = ingredient.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
/var/folders/c5/pphx1c0n3lb12sy85_ybqxqm0000gn/T/ipykernel_28708/2816357255.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ingredient["Description"] = ingredient["Description"].str.replace("nan","")
/var/folders/c5/pphx1c0n3lb12sy85_ybqxqm0000gn/T/ipykernel_28708/2816357255.py:

,ItemId,Description,Qty,UOM,PrepId
0,I-0,Radish - Daikon,1000.000,g,P-0
1,I-1,2022.3 Basic Pickling Liqui d,1000.000,g,P-0
0,I-2,NORI KIZAMI BTL,1.000,g,P-1
1,I-3,PORT PORT 2022.3 Double Down Chick e,2.000,PORT,P-1
2,I-4,PATTY PATTY 2022.3 Rice Patty Cake,1.000,PATTY,P-1


In [19]:
ams_items.loc[ams_items["PrepId"] == "P-24"]

,ItemId,Description,Qty,UOM,PrepId
0,I-132,LBS LBS 2022.3 Marinated Chicken T,10.000,LBS,P-24


In [282]:
def define_dependent_items(df):
    condition = df["Description"].str.contains("2022.3")
    dependent_items = df[condition]
    return dependent_items

define_dependent_items(ams_items)

,ItemId,Description,Qty,UOM,PrepId
1,I-1,2022.3 Basic Pickling Liqui d,1000.000,g,P-0
1,I-3,PORT PORT 2022.3 Double Down Chick e,2.000,PORT,P-1
2,I-4,PATTY PATTY 2022.3 Rice Patty Cake,1.000,PATTY,P-1
0,I-39,2022.3 Cooked Penne,10.000,oz,P-8
1,I-40,2022.3 Marinara,6.000,oz,P-8
6,I-50,2022.3 Thyme Oil,1000.000,g,P-9
2,I-73,2022.3 Smoked Paprika Ru,30.000,g,P-14
0,I-88,2022.3 Potato Patty,120.000,g,P-17
1,I-99,LBS LBS 2022.3 Marinated Chicken T,10.000,LBS,P-19
0,I-103,PATTY PATTY 2022.3 Rice Patty Cake,2.000,PATTY,P-21


In [271]:
mask = ams_items["Description"].str.contains('2022.3')
ams_masked = ams_items[mask]
print(len(ams_masked))
ams_masked["OriginId"] = ""
ams_masked

12


/var/folders/c5/pphx1c0n3lb12sy85_ybqxqm0000gn/T/ipykernel_6808/2486805572.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ams_masked["OriginId"] = ""


,ItemId,Description,Qty,UOM,PrepId,OriginId
1,I-1,2022.3 Basic Pickling Liqui d,1000.000,g,P-0,
1,I-3,PORT PORT 2022.3 Double Down Chick e,2.000,PORT,P-1,
2,I-4,PATTY PATTY 2022.3 Rice Patty Cake,1.000,PATTY,P-1,
0,I-39,2022.3 Cooked Penne,10.000,oz,P-8,
1,I-40,2022.3 Marinara,6.000,oz,P-8,
6,I-50,2022.3 Thyme Oil,1000.000,g,P-9,
2,I-73,2022.3 Smoked Paprika Ru,30.000,g,P-14,
0,I-88,2022.3 Potato Patty,120.000,g,P-17,
1,I-99,LBS LBS 2022.3 Marinated Chicken T,10.000,LBS,P-19,
0,I-103,PATTY PATTY 2022.3 Rice Patty Cake,2.000,PATTY,P-21,


In [263]:
prep_num = 0

def generate_products():
    global products_list
    columns = ["Description"]
    df = pd.DataFrame(products_list, columns=columns)
    df["PrepId"] = ""
    
    for ind, row in df.iterrows():

        global prep_num
        df.loc[ind, "PrepId"] = "P-" + str(prep_num)
        prep_num += 1
    
    df['Description'] = df['Description'].str.replace('^2022.3 ','')
    df['Description'] = df['Description'].str.replace('\s*FLAVOUR LAB \(FLAVLAB\)','')
#     df['Description'] = df['Description'].str.replace('\w*COMMISSARY','')
    prep_col_list = ["PrepId", "Description"]
    df = df[prep_col_list]
    
    return df
    
prod_df = generate_products()
prod_df

/var/folders/c5/pphx1c0n3lb12sy85_ybqxqm0000gn/T/ipykernel_6808/1881777914.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Description'] = df['Description'].str.replace('^2022.3 ','')
/var/folders/c5/pphx1c0n3lb12sy85_ybqxqm0000gn/T/ipykernel_6808/1881777914.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Description'] = df['Description'].str.replace('\s*FLAVOUR LAB \(FLAVLAB\)','')


,PrepId,Description
0,P-0,Pickled Daikon
1,P-1,Full KFC Double Down
2,P-2,Smoked Tofu Block
3,P-3,Pesto Cream
4,P-4,Smoked Paprika Rub
5,P-5,Onion/Garlic/Oil
6,P-6,Marinara SauceCOMMISSARY (COM)
7,P-7,Potato Skins
8,P-8,Cheap Pasta
9,P-9,Mixed Mushroom


In [67]:
dep_items = pd.read_csv("notebooks/data/AMS/preprocessed/Dependent_Items.csv")
dep_items

,ItemId,Description,Qty,UOM,PrepId,OriginId
0,I-1,2022.3 Basic Pickling Liqui d,1000.00,g,P-0,NaN
1,I-3,PORT PORT 2022.3 Double Down Chick e,2.00,PORT,P-1,P-19
2,I-4,PATTY PATTY 2022.3 Rice Patty Cake,1.00,PATTY,P-1,P-20
3,I-39,2022.3 Cooked Penne,10.00,oz,P-8,NaN
4,I-40,2022.3 Marinara,6.00,oz,P-8,NaN
5,I-50,2022.3 Thyme Oil,1000.00,g,P-9,P-25
6,I-73,2022.3 Smoked Paprika Ru,30.00,g,P-14,P-4
7,I-88,2022.3 Potato Patty,120.00,g,P-17,P-11
8,I-99,LBS LBS 2022.3 Marinated Chicken T,10.00,LBS,P-19,P-26
9,I-103,PATTY PATTY 2022.3 Rice Patty Cake,2.00,PATTY,P-21,P-20


In [76]:
dep_list = dep_items.dropna(subset=['OriginId'])
dep_list2 = dep_items["OriginId"].unique().tolist()
dup_list = []

for item in dep_list2:
    if item in dep_items["PrepId"].tolist():
        dup_list.append(item)
    else:
        continue
        
dup_list

['P-19']

In [104]:
col_list = ["ItemId", "Description", "Qty", "UOM", "PrepId"]
new_df = pd.DataFrame(columns=col_list)

for item in dep_items["OriginId"].unique().tolist():
    for ind, row in ams_items.iterrows():
        if row["PrepId"] == item:
            new_df = new_df.append(pd.Series(row))

new_df = new_df.reset_index()
            
for ind, row in new_df.iterrows():
    if row["PrepId"] in dup_list:
        new_df = new_df.drop(ind)
        
new_df = new_df.drop(["index"], axis=1)
print(len(new_df))
new_df

23


,ItemId,Description,Qty,UOM,PrepId
2,I-100,2022.2 Cooked Sushi Rice,1350.000,g,P-20
3,I-101,SHICHIMI TOGARASHI,15.000,g,P-20
4,I-102,PFO OIL CANOLA JIB AN T,100.000,ml,P-20
5,I-133,PFO OIL CANOLA JIB AN T,3000.000,ml,P-25
6,I-134,THYME,50.000,g,P-25
7,I-15,SPICE SMOKED PAPRIKA,250.000,g,P-4
8,I-16,SPICE CHILI POWDER,50.000,g,P-4
9,I-17,Spice-Seasoning Salt Lawr y,600.000,g,P-4
10,I-18,SUGAR GRANULATED FI N,100.000,g,P-4
11,I-55,POTATO RUSSET BAKER,3250.000,g,P-11


In [98]:
new_df.to_csv(f"notebooks/data/AMS/preprocessed/Independent_items{str(datetime.date(datetime.now()))}.csv")

In [101]:
df = pd.read_csv("notebooks/data/AMS/preprocessed/Categaroy_Assigned.csv")
df
print(df.dtypes)

ConversionId       object
Multiplier        float64
ConvertFromQty    float64
ConvertFromUom     object
ConvertToQty      float64
ConvertToUom       object
Category            int64
dtype: object


In [34]:
labelled = pd.read_csv("notebooks/data/AMS/preprocessed/Independent_items_labelled.csv")
print(len(labelled))
labelled

26


,ProdId,Weight (g),Total CO2 Emission (g),Total Nitrogen Emission (g),Total Freshwater Withdrawals (mL),GHG Emission (g) / 100g,N lost (g) / 100g,Stress-Weighted Water Use (L) / 100g
0,P-0,1000.000,306.200000,7.900000,9.900000e+03,30.620000,0.790000,0.990000
1,P-17,15.000,4.593000,0.118500,1.485000e+02,30.620000,0.790000,0.990000
2,P-23,28610.560,1177.804032,7.920492,1.038172e+05,4.116676,0.027684,0.362863
3,P-1,467.500,24.390650,0.686900,4.853175e+03,5.217251,0.146930,1.038112
4,P-6,32017.800,4871.589038,21.982096,6.135616e+04,15.215252,0.068656,0.191631
5,P-9,165133.120,10405.400756,161.646956,1.663834e+06,6.301220,0.097889,1.007572
6,P-22,204440.000,62220.395100,482.925700,4.618590e+06,30.434551,0.236219,2.259142
7,P-25,6100.000,9745.445000,0.995000,8.265000e+03,159.761393,0.016311,0.135492
8,P-8,19.680,44.563924,0.151574,2.108280e+02,226.442703,0.770193,1.071280
9,P-11,33705.000,1843.916700,20.735700,8.981210e+04,5.470751,0.061521,0.266465


In [33]:
double_list = dep_items["OriginId"].unique().tolist()
double_list = [x for x in double_list if x == x]

for ind, row in labelled.iterrows():
    
    if row["ProdId"] in double_list:
        labelled = labelled.drop(ind)
        
labelled

,ProdId,Weight (g),Total CO2 Emission (g),Total Nitrogen Emission (g),Total Freshwater Withdrawals (mL),GHG Emission (g) / 100g,N lost (g) / 100g,Stress-Weighted Water Use (L) / 100g
0,P-0,1000.00,306.200000,7.900000,9900.000,30.620000,0.790000,0.990000
1,P-17,15.00,4.593000,0.118500,148.500,30.620000,0.790000,0.990000
2,P-23,28610.56,1177.804032,7.920492,103817.154,4.116676,0.027684,0.362863
3,P-1,467.50,24.390650,0.686900,4853.175,5.217251,0.146930,1.038112
4,P-6,32017.80,4871.589038,21.982096,61356.159,15.215252,0.068656,0.191631
5,P-9,165133.12,10405.400756,161.646956,1663834.332,6.301220,0.097889,1.007572
6,P-22,204440.00,62220.395100,482.925700,4618589.691,30.434551,0.236219,2.259142
8,P-8,19.68,44.563924,0.151574,210.828,226.442703,0.770193,1.071280
10,P-18,7972.80,1017.857140,3.734640,39120.580,12.766621,0.046842,0.490676
11,P-21,1674.60,122.353390,2.075140,18646.455,7.306425,0.123919,1.113487


In [265]:
diff_df = pd.merge(ams_items, ams_masked, how='outer', indicator='Exist')
diff_df = diff_df.loc[diff_df['Exist'] != 'both']
diff_df = diff_df.drop(["Exist"], axis=1)
diff_df.to_csv("notebooks/data/AMS/preprocessed/Independent_Items.csv", index=False)
diff_df

,ItemId,Description,Qty,UOM,PrepId
0,I-0,Radish - Daikon,1000.000,g,P-0
2,I-2,NORI KIZAMI BTL,1.000,g,P-1
5,I-5,Lettuce - Butter,0.050,ea,P-1
6,I-6,TOMATO - 6 X 7 MED,30.000,g,P-1
7,I-7,2022 KFC Sauce,45.000,g,P-1
...,...,...,...,...,...
131,I-131,SUGAR GRANULATED FI N,15.000,g,P-23
133,I-133,PFO OIL CANOLA JIB AN T,3000.000,ml,P-25
134,I-134,THYME,50.000,g,P-25
135,I-135,Chicken Flat Thigh s/onknu c,50.000,lb,P-26


In [255]:
ams_items.loc[ams_items["PrepId"] == "P-9"]

,ItemId,Description,Qty,UOM,PrepId
0,I-44,MUSHROOM,20.000,lb,P-9
1,I-45,"MUSHROOM ""PORTABEL L",5.000,lb,P-9
2,I-46,Mushroom Cremini,5.000,lb,P-9
3,I-47,Mushrooms Oyster,5.000,lb,P-9
4,I-48,SALT COARSE KOSHER,150.000,g,P-9
5,I-49,Black Pepper Fine,30.000,g,P-9
6,I-50,2022.3 Thyme Oil,1000.000,g,P-9
7,I-51,THYME,50.000,g,P-9
8,I-52,MUSHROOMS WHT SLCD,10.000,lb,P-9


In [9]:
temp.to_csv("notebooks/data/AMS/preprocessed/AMS_Ingredients_List.csv", index=False)

In [268]:
products_list

['2022.3 Pickled DaikonFLAVOUR LAB (FLAVLAB)',
 '2022.3 Full KFC Double DownFLAVOUR LAB (FLAVLAB)',
 '2022.3 Smoked Tofu BlockFLAVOUR LAB (FLAVLAB)',
 '2022.3 Pesto CreamFLAVOUR LAB (FLAVLAB)',
 '2022.3 Smoked Paprika RubFLAVOUR LAB (FLAVLAB)',
 '2022.3 Onion/Garlic/OilFLAVOUR LAB (FLAVLAB)',
 'Marinara SauceCOMMISSARY  (COM)',
 '2022.3 Potato Skins FLAVOUR LAB (FLAVLAB)',
 '2022.3 Cheap PastaFLAVOUR LAB (FLAVLAB)',
 '2022.3 Mixed MushroomFLAVOUR LAB (FLAVLAB)',
 '2022.3 Confit Garlic OilFLAVOUR LAB (FLAVLAB)',
 '2022.3 Potato PattyFLAVOUR LAB (FLAVLAB)',
 '2022.3 Gochujang Alfredo FLAVOUR LAB (FLAVLAB)',
 '2022.3 Cooked BowtieFLAVOUR LAB (FLAVLAB)',
 '2022.3 Fried Chickpea FLAVOUR LAB (FLAVLAB)',
 '2022.3 GF KFC Sauce FLAVOUR LAB (FLAVLAB)',
 '2022.3 Sumac DressingFLAVOUR LAB (FLAVLAB)',
 '2022.3 Aloo Tikki FLAVOUR LAB (FLAVLAB)',
 '2022.3 Roasted Veggie Mix FLAVOUR LAB (FLAVLAB)',
 '2022.3 Double Down Chicken FLAVOUR LAB (FLAVLAB)',
 '2022.3 Rice Patty CakeFLAVOUR LAB (FLAVLAB)',
 '2

In [133]:
prod_df.to_csv("notebooks/data/AMS/preprocessed/AMS_Products_List.csv", index=False)

In [186]:
ams_items["CaseUOM"].unique()

array(['g', 'PORT', 'PATTY', 'ea', 'lb', 'L', 'ml', 'BUNC', 'fl', '',
       'slice', 'oz', 'Kg', 'LBS'], dtype=object)

In [290]:
recipe_list = {"Full KFC Double Down": ["Double Down Chicken", "Marinated Chicken Thigh",
                                         "Rice Patty Cake", "Pickled Daikon", "GF KFC Sauce"],
              "Sauce Gochujang Alfredo": ["Gochujang Alfredo", "PREP SOUR CREAM"],
              " Aloo Tikki Burger": ["Full Aloo Tikki Burger", "Aloo Tikki", "Potato Patty", "Mint Chutney"],
              "Rice Cake Burger": ["Full Smoked Tofu Burger", "Rice Patty Cake", "Smoked Tofu Block", "Sesame Sweet Soy"],
              "Fattoush Salad": ["Fattoush Salad", "Fried Chickpea", "Sumac Dressing"],
              "Sauce Gochujang Alfredo": ["Gochujang Alfredo", "PREP SOUR CREAM ALFREDO"],
              "Sauce Allio e ollio":["Aglio e olio", "Onion/Garlic/Oil", "Confit Garlic Oil"]}
for item in recipe_list.items():
    print(item[1])

['Double Down Chicken', 'Marinated Chicken Thigh', 'Rice Patty Cake', 'Pickled Daikon', 'GF KFC Sauce']
['Gochujang Alfredo', 'PREP SOUR CREAM ALFREDO']
['Full Aloo Tikki Burger', 'Aloo Tikki', 'Potato Patty', 'Mint Chutney']
['Full Smoked Tofu Burger', 'Rice Patty Cake', 'Smoked Tofu Block', 'Sesame Sweet Soy']
['Fattoush Salad', 'Fried Chickpea', 'Sumac Dressing']
['Aglio e olio', 'Onion/Garlic/Oil', 'Confit Garlic Oil']


In [298]:
list(recipe_list.values())

[['Double Down Chicken',
  'Marinated Chicken Thigh',
  'Rice Patty Cake',
  'Pickled Daikon',
  'GF KFC Sauce'],
 ['Gochujang Alfredo', 'PREP SOUR CREAM ALFREDO'],
 ['Full Aloo Tikki Burger', 'Aloo Tikki', 'Potato Patty', 'Mint Chutney'],
 ['Full Smoked Tofu Burger',
  'Rice Patty Cake',
  'Smoked Tofu Block',
  'Sesame Sweet Soy'],
 ['Fattoush Salad', 'Fried Chickpea', 'Sumac Dressing'],
 ['Aglio e olio', 'Onion/Garlic/Oil', 'Confit Garlic Oil']]

In [301]:
ams_prep = pd.read_csv("notebooks/data/AMS/preprocessed/Independent_Items.csv")
ams_prep["Recipe"] = ""
ams_prep

,ItemId,Description,Qty,UOM,PrepId,Recipe
0,I-0,Radish - Daikon,1000.00,g,P-0,
1,I-2,NORI KIZAMI BTL,1.00,g,P-1,
2,I-5,Lettuce - Butter,0.05,ea,P-1,
3,I-6,TOMATO - 6 X 7 MED,30.00,g,P-1,
4,I-7,2022 KFC Sauce,45.00,g,P-1,
...,...,...,...,...,...,...
120,I-131,SUGAR GRANULATED FI N,15.00,g,P-23,
121,I-133,PFO OIL CANOLA JIB AN T,3000.00,ml,P-25,
122,I-134,THYME,50.00,g,P-25,
123,I-135,Chicken Flat Thigh s/onknu c,50.00,lb,P-26,


In [13]:
os.getcwd()

'/Users/jennylee/CFFS-PyCharm/notebooks'

In [14]:
dep_items = pd.read_csv("data/AMS/preprocessed/Dependent_Items.csv")
dep_items

,ItemId,Description,Qty,UOM,PrepId,OriginId
0,I-1,2022.3 Basic Pickling Liqui d,1000.00,g,P-0,NaN
1,I-3,PORT PORT 2022.3 Double Down Chick e,2.00,PORT,P-1,P-19
2,I-4,PATTY PATTY 2022.3 Rice Patty Cake,1.00,PATTY,P-1,P-20
3,I-39,2022.3 Cooked Penne,10.00,oz,P-8,NaN
4,I-40,2022.3 Marinara,6.00,oz,P-8,NaN
5,I-50,2022.3 Thyme Oil,1000.00,g,P-9,P-25
6,I-73,2022.3 Smoked Paprika Ru,30.00,g,P-14,P-4
7,I-88,2022.3 Potato Patty,120.00,g,P-17,P-11
8,I-99,LBS LBS 2022.3 Marinated Chicken T,10.00,LBS,P-19,P-26
9,I-103,PATTY PATTY 2022.3 Rice Patty Cake,2.00,PATTY,P-21,P-20


In [304]:
organize_df = pd.DataFrame(columns=["Child", "Product"])

for recipe in recipe_list.items():
    for ind, row in prod_df.iterrows():
        if row["Description"] == recipe:
            product = row["PrepId"]
    
        ingre = recipe[1]

        if row["Description"] in ingre:
            recipe_id = 

['Double Down Chicken', 'Marinated Chicken Thigh', 'Rice Patty Cake', 'Pickled Daikon', 'GF KFC Sauce']
PrepId                    P-0
Description    Pickled Daikon
Name: 0, dtype: object
PrepId                 P-15
Description    GF KFC Sauce
Name: 15, dtype: object
PrepId                        P-19
Description    Double Down Chicken
Name: 19, dtype: object
PrepId                    P-20
Description    Rice Patty Cake
Name: 20, dtype: object
PrepId                            P-26
Description    Marinated Chicken Thigh
Name: 26, dtype: object
['Gochujang Alfredo', 'PREP SOUR CREAM ALFREDO']
PrepId                      P-12
Description    Gochujang Alfredo
Name: 12, dtype: object
['Full Aloo Tikki Burger', 'Aloo Tikki', 'Potato Patty', 'Mint Chutney']
PrepId                 P-11
Description    Potato Patty
Name: 11, dtype: object
PrepId               P-17
Description    Aloo Tikki
Name: 17, dtype: object
PrepId                 P-23
Description    Mint Chutney
Name: 23, dtype: object
['F

In [303]:
prod_df

,PrepId,Description
0,P-0,Pickled Daikon
1,P-1,Full KFC Double Down
2,P-2,Smoked Tofu Block
3,P-3,Pesto Cream
4,P-4,Smoked Paprika Rub
5,P-5,Onion/Garlic/Oil
6,P-6,Marinara SauceCOMMISSARY (COM)
7,P-7,Potato Skins
8,P-8,Cheap Pasta
9,P-9,Mixed Mushroom


In [289]:
# Try to assign values for a burger/pasta. 
prod_num = 0

for ind, row in prod_df.iterrows():
    for item in recipe_list.items():
        if row["Description"] in item[1]:
            ams_prep.loc[ind, "Recipe"] = item[0]
        else:
            continue
            
ams_prep

,ItemId,Description,Qty,UOM,PrepId,Recipe
0,I-0,Radish - Daikon,1000.00,g,P-0,
1,I-2,NORI KIZAMI BTL,1.00,g,P-1,
2,I-5,Lettuce - Butter,0.05,ea,P-1,
3,I-6,TOMATO - 6 X 7 MED,30.00,g,P-1,
4,I-7,2022 KFC Sauce,45.00,g,P-1,
...,...,...,...,...,...,...
120,I-131,SUGAR GRANULATED FI N,15.00,g,P-23,
121,I-133,PFO OIL CANOLA JIB AN T,3000.00,ml,P-25,
122,I-134,THYME,50.00,g,P-25,
123,I-135,Chicken Flat Thigh s/onknu c,50.00,lb,P-26,


In [15]:
ams_initial = pd.read_csv("match_final.csv")
ams_initial.head(30)

,ProdId,Weight (g),Total CO2 Emission (g),Total Nitrogen Emission (g),Total Freshwater Withdrawals (mL),Total CO2 Emission (g) (100),Total Nitrogen Emission (g) (100),Total Freshwater Withdrawals (L) (100)
0,P-0,1000.000,306.200000,7.900000,9.900000e+03,30.620000,0.790000,0.990000
1,P-17,15.000,4.593000,0.118500,1.485000e+02,30.620000,0.790000,0.990000
2,P-23,28610.560,1177.804032,7.920492,1.038172e+05,4.116676,0.027684,0.362863
3,P-1,467.500,24.390650,0.686900,4.853175e+03,5.217251,0.146930,1.038112
4,P-6,32017.800,4871.589038,21.982096,6.135616e+04,15.215252,0.068656,0.191631
5,P-9,165133.120,10405.400756,161.646956,1.663834e+06,6.301220,0.097889,1.007572
6,P-22,204440.000,62220.395100,482.925700,4.618590e+06,30.434551,0.236219,2.259142
7,P-25,6100.000,9745.445000,0.995000,8.265000e+03,159.761393,0.016311,0.135492
8,P-8,19.680,44.563924,0.151574,2.108280e+02,226.442703,0.770193,1.071280
9,P-11,33705.000,1843.916700,20.735700,8.981210e+04,5.470751,0.061521,0.266465


In [139]:
os.getcwd()

'/Users/jennylee/CFFS-PyCharm'

In [140]:
ams_nonstd = pd.read_csv("notebooks/data/AMS/conversions/NonStd_Units.csv")
ams_nonstd

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,I-3,166.50,6.00,PORT,999.0,999
1,I-4,333.00,3.00,PATTY,999.0,999
2,I-5,6660.00,0.15,ea,999.0,999
3,I-12,333.00,3.00,ea,999.0,999
4,I-15,6660.00,0.15,ea,999.0,999
5,I-19,333.00,3.00,PATTY,999.0,999
6,I-29,33300.00,0.03,BUNC,999.0,999
7,I-30,333.00,3.00,ea,999.0,999
8,I-38,999.00,1.00,BUNC,999.0,999
9,I-45,83.25,12.00,ea,999.0,999
